# Spike Tutorial -- Part 2

## Analyzing Populations 1

In part 1 we had to do quite a bit of work to get our data in a workable format. Now that we understand the data structures and how to manipulate them, we can focus more on scientific questions. In this tutorial, the focus will be on analyzing multiple neurons and organizing and analyzing that data. There are many ways to do this in Matlab, and it seems like even more in Python, so we'll explore a few of them.

First import our favorite modules!

In [ ]:
# import necessary modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle

%matplotlib inline

We pickled our session info. from last time, let's load it up:

In [ ]:
root = '_data'
fn = os.path.join(root,'sessionInfo.pkl')

with open(fn,'rb') as f:
    [order, clustInfo, fs, stimOn, laserOn] = pickle.load(f)

Ok, we have almost everything we need except our spikes. Load the `'spike_times.npy'` and `'spike_clusters.npy'` to `spikes` and `clust`, respectively. Convert your spikes to seconds using `fs`.

In [ ]:
# load spikes and clusters


We have all of our raw data, lets use what we learned from last time to make PSTHs and rasters for each cell! Because we need to make a PSTH for every neuron, we should write a function to do that. Write a function called `makePSTH` that takes 3 inputs: `spikes`, `triggers`, `edges` and returns `psth`, `raster` and `trials`.

TIP: make the function as general as possible... to compute the firing rate per bin you need to divide by the bin time. Is there a way to do that from just the three variables we've passed?

In [ ]:
# makePSTH function

        

Like last time, lets define `edges` in steps of .001 between -.1 and 3.1s, and then define a vector called `time` with the edge centers. Take some spikes from unit 26 and use your function to make `psth`, `raster`, and `trials`. Scatter plot the raster to make sure this worked well.

In [ ]:
# compute psth, raster
c = 26


# plot


Next we want to compute PSTHs for all of our neurons and save them to a data struct. There are a lot of ways to do this, but the way we can first try is by saving them in a 3d array (that is, n trials x m timepoints x p neurons).

**Try this:**
1. Make an **array** called `cellID` that contains the unique cell ID numbers for all our our neurons in `clustInfo`.
2. Loop through this variable. On each loop, extract spikes for that cell and use your PSTH function to make a PSTH (don't make raster,trials for now)
3. Save each cell's PSTH to a new matrix called `allPSTH`, like described above. Remember to preallocate an **empty** version of this matrix.
4. This might take a little while to run, print the iteration each time the loop runs to keep track of your progress.

In [ ]:
# answer here

The benefit of putting everything in an array is that we can really easily index and average data using numpy. 

**Try this:**
1. As a first pass, make an index for laser ON/OFF trials.
2. Then index `allPSTH` and average to find the mean response over all neurons when the laser is on vs when it is off. You can save your responses to `meanLaserOff` and `meanLaserOn`. HINT: you need to average over more than 1 dimension!
3. Plot these cell averages on top of each other. Give them a legend and axis labels.

In [ ]:
# answer here

Woah! It should look pretty funky, with the laserOn plot having very stereotyped bursting activity. This is because the laser was pulsed at 25Hz and it looks like there are some optotagged units in this recording! (that means that the unit will be activated on each laser pulse).

Lets try to separate these units out from the rest.

**Try this:**
1. Compute new averages, averaging over the entire trace on laser off vs laser on trials for each cell. Your ending vectors will be length of the number of neurons, 49. Call them `cellLaserOff` and `cellLaserOn`.
2. Compute a difference vector, subtracting the firing rate when the laser is off from when it is on, call this `dLaser`

In [ ]:
# compute the average response difference between laser off and laser on trials for each neuron


Now you can use `dLaser` as an index to separate cells that are activated by the laser vs suppressed by the laser.

1. Make two new PSTHs, one called `supPSTH` which contains neurons that are suppressed by laser, and another called `actPSTH` which contains only activated neurons.
2. Get the population means for laser on and off trials for each of these sets of neurons.
3. On two subplots, plot laser on vs laser off traces for each population. Color the activated subplot in black for laser off and black, dashed lines for on. Do the same for the suppressed neurons subplot, but in blue.

In [ ]:
# answer here

This simple way of filtering the data looks like it did an ok job of filtering out the different cell types. However, it's not perfect (see in the laser on condition in suppressed cells, there is still some laser-locked activity). Maybe we can try to refine this method a bit by using statistical tests to tell whether a cell significantly changes its activity.

We can do this by using a ranksum test for each cell, across all trials, and then asking whether the difference is significant, and in what direction. We can use the ranksums function in the scipy.stats module to do this. To run the test, lets just look at the time window from 0 to 1s.

In [ ]:
import scipy.stats

c = 32

# answer here

Here, f and p are results of the test. In our case, f is the test statistic, and its sign will indicate whether the cell was activated by laser (positive f) or suppressed (negative). The p value tells us if our cell was significantly modulated by laser.

That said, we want information for each cell. Use a for loop to calculate the test statistics and p-values for all the neurons. Save your results to variables called `fVal` and `pVal`.

In [ ]:
# answer here

Now we have another cell selection criteria we can use, whether the cell was significantly modulated by laser. However, we just ran quite a few tests, by chance some of the tests will be false positives. There are many ways to correct for this, but lets use the simplest one that is also quite strict, the Bonferonni correction, which just states that our new significance value is our previous criterion for significance, divided by the number of tests we ran.

In [ ]:
alpha = .05 / len(cellID)
print(alpha)

Using this new criterion, nearly all of the cells are still significant (this is probably due to the large number of trials we had). By my count, only one cell failed the test, lets look at that cell:

**try this:**
1. Make an index that finds cells violating our corrected alpha level, call it `pI`.
2. Plot the average traces for this cell, does the test result make sense to you?

In [ ]:
# answer here

It looks like that previous cell didn't have much activity, so it probably had quite a bit of variability over trials that contributed to it failing the test.

So far, we've organized our data in arrays, but once we get to this population level, maybe panda dataframes are a useful way to organize and select neurons for analysis, we'll try this in the next tutorial.